In [2]:
import os
import PyPDF2
import random

def extract_text_from_pdfs(pdf_folder):
    all_text = ''
    for filename in os.listdir(pdf_folder):
        if filename.endswith('.pdf'):
            path = os.path.join(pdf_folder, filename)
            with open(path, 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                for page in reader.pages:
                    all_text += page.extract_text() + '\n'
    return all_text

pdf_folder = './pdfs'  # altere para o caminho real
corpus_text = extract_text_from_pdfs(pdf_folder)

In [3]:
import re
import nltk
# from nltk.tokenize import sent_tokenize

# nltk.download('punkt',download_dir='./nltk_data', quiet=True)

def preprocess_text(text):
    text = text.lower()
    # text = re.sub(r'\n+', ' ', text)
    # # MANTÉM . ! ? para detecção de fim de frase
    # text = re.sub(r'[^a-záéíóúàãõç\.\!\?\s]', '', text)
    # return text
    text = text.replace('\xa0', ' ')
    text = re.sub(r'\n+', '\n', text)  # mantém quebras de parágrafo
    text = re.sub(r'[ \t]+', ' ', text)
    text = re.sub(r' +\n', '\n', text)
    return text.strip()

# corpus_text = "Belém é a capital do Pará. É conhecida pelo Círio de Nazaré! Você já visitou?"
processed_text = preprocess_text(corpus_text)
# clean_text = sent_tokenize(processed_text, language='portuguese')

In [7]:
from tensorflow.keras.models import load_model

# Load the model
model = load_model('portuguese_chatbot.h5')

# Verify the model is loaded
model.summary()

import pickle

# Carrega o arquivo pickle
with open('tokenizer.pickle', 'rb') as file:
    data = pickle.load(file)

print("Conteúdo carregado:", type(data))

tokenizer = data

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 25, 128)        │        47,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 25, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 25, 512)        │       788,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 25, 512)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 25, 512)        │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 25, 512)        │     1,574,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 25, 512)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 25, 512)        │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 25, 370)        │       189,810 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,605,172 (9.94 MB)

 Trainable params: 2,602,866 (9.93 MB)

 Non-trainable params: 2,304 (9.00 KB)

 Optimizer params: 2 (12.00 B)

Conteúdo carregado: <class 'keras.src.legacy.preprocessing.text.Tokenizer'>


In [9]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization, Bidirectional, RepeatVector
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import re
import unicodedata



# Preprocessing functions
def preprocess_text(text):
    # Normalize and clean text
    text = unicodedata.normalize('NFKD', text.lower()).encode('ASCII', 'ignore').decode('ASCII')
    text = re.sub(r'[^\w\s]', '', text)
    return text

max_length=25

def generate_response(input_text, temperature=0.8):
    """
    Generate a response to the input text using the trained model.
    
    Args:
        input_text: User input string
        temperature: Controls randomness of predictions (higher = more random)
        
    Returns:
        Generated response string
    """
    # Preprocess input
    cleaned_input = preprocess_text(input_text)
    input_seq = tokenizer.texts_to_sequences([cleaned_input])
    input_padded = pad_sequences(input_seq, maxlen=max_length, padding='post')
    
    # Initialize response
    response_seq = np.zeros((1, max_length))
    response_seq[0, 0] = tokenizer.word_index['<OOV>']  # Start token
    
    for i in range(1, max_length):
        # Predict next word
        predictions = model.predict([input_padded, response_seq], verbose=0)[0][i-1]
        
        # Apply temperature for diversity
        predictions = np.log(predictions) / temperature
        exp_preds = np.exp(predictions)
        predictions = exp_preds / np.sum(exp_preds)
        
        # Sample from predictions
        next_word_idx = np.random.choice(len(predictions), p=predictions)
        response_seq[0, i] = next_word_idx
        
        # Stop if end token is predicted
        if next_word_idx == 0:
            break
    
    # Convert sequence to text
    response_tokens = []
    for idx in response_seq[0]:
        if idx == 0:  # Skip padding
            continue
        word = tokenizer.index_word.get(idx, '')
        if word == '<OOV>':  # Skip OOV tokens
            continue
        response_tokens.append(word)
    
    return ' '.join(response_tokens)

# Test the chatbot
test_inputs = [
    "oi",
    "conte-me sobre o folclore",
    "quais frutas tem na amazônia?",
    "quem é o curupira?",
    "o que é Ver-o-Peso"
]

for input_text in test_inputs:
    response = generate_response(input_text)
    print(f"User: {input_text}")
    print(f"Chatbot: {response}\n")

/home/prinako/External/School/UFPA/ITEC_UFPA/24.4_SEMESTER/INTELIGENCIA_COMPUTACIONAL/T2/amazon-portuguese-chatbot/venv/lib/python3.11/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: input_layer
Received: inputs=('Tensor(shape=(1, 25))', 'Tensor(shape=(1, 25))')
  warnings.warn(msg)
I0000 00:00:1743513961.650349  804478 cuda_dnn.cc:529] Loaded cuDNN version 90300


User: oi
Chatbot: ajudar 40000 de internacional na pessoas lenda amazonicas do sabores unica hoje quais borboletas catirina conversar quente oi quer conhecer pato mandioca hidrovia cultura

User: conte-me sobre o folclore
Chatbot: 18351840 revolta 26c belem tacaca mandioca amazonia vermelhos oncas aqui outubro morte amazonia teve xingu milhao conta capital roteiro boi os 3 carnes equatorial

User: quais frutas tem na amazônia?
Chatbot: todo morte voce namerica culinaria amazonica prato portuguesas de vermelhos fundada cozido passaros quem francisco valdecans resistencia catirina fundada porto 26c liquido boibumba brasil

User: quem é o curupira?
Chatbot: explosao docas chamada borboletas das cidade reconhecida segundo 4 carnes tacaca 4 simbolo antecedencia tucupi m2 mangal comidas falar voos mortos principais cabelos m2

User: o que é Ver-o-Peso
Chatbot: regiao uma comidas chamada floresta construido considerada fascinante noite 12 cezar peixe 18791912 folclore chuvas destes verequete 

In [ ]:
while True:
    input_text = input("User: ")
    response = generate_response(input_text)
    print(f"Chatbot: {response}\n")

Chatbot: incriveis carnes 10 carimbo a ficar mercado garcas acontece goeldi aqui carne do fauna folclorica historico destacamse ecologico unica lingua teatro folclores culinaria folclore

Chatbot: macacos 1625 e internacional neoclassico junho clima seco rios explosao pontos amazonia cabelos sopa clima biodiversidade famoso marujada

Chatbot: melhor marujada visitar guama quem dos falar incriveis emilio chamada pobreza turisticas verequete jari destes quando folclore abriga boa cezar hidrovia incriveis tapajos domingo

